<a href="https://colab.research.google.com/github/TONNY-01/Original/blob/main/Make_Automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import os
import cloudinary
import cloudinary.uploader
import requests
from PIL import Image
import io
from datetime import datetime

import cloudinary
import cloudinary.uploader
from cloudinary.utils import cloudinary_url
from diffusers import StableDiffusionXLPipeline, DPMSolverMultistepScheduler
from compel import Compel, ReturnedEmbeddingsType
import torch

# Read input parameters
with open('../input/stable-diffusion-input-*/input.json', 'r') as f:
    params = json.load(f)

prompt = params['prompt']
num_images = params['num_images']
model_id = params['model_id']
height = params['height']
width = params['width']

device = 'cuda'
pipeline = StableDiffusionXLPipeline.from_pretrained(
    model_id, torch_dtype=torch.float16).to(device)
pipeline.safety_checker = None

compel = Compel(tokenizer=[pipeline.tokenizer, pipeline.tokenizer_2],
                text_encoder=[pipeline.text_encoder, pipeline.text_encoder_2],
                returned_embeddings_type=ReturnedEmbeddingsType.PENULTIMATE_HIDDEN_STATES_NON_NORMALIZED,
                requires_pooled=[False, True])

pipeline.scheduler = DPMSolverMultistepScheduler.from_config(pipeline.scheduler.config)

neg = "lowres, mosaic censoring, bar censor, stripe, censored, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry"
steps = 30
guidance = 7.5

conditioning, pooled = compel([prompt, neg])

images = pipeline(prompt_embeds=conditioning[0:1], pooled_prompt_embeds=pooled[0:1],
                  negative_prompt_embeds=conditioning[1:2], negative_pooled_prompt_embeds=pooled[1:2],
                  height=height, width=width, num_inference_steps=steps, guidance_scale=guidance,
                  num_images_per_prompt=num_images, use_karras_sigmas=True).images

# Configuration
cloudinary.config(
    cloud_name = "djc6qqghd",
    api_key = "272687192445414",
    api_secret = "CCWiccNQ5_5U7vELMpBVtVu7RXE",
    secure=True
)

def upload_to_cloudinary(image_path):
    response = cloudinary.uploader.upload(image_path)
    return response['secure_url']

for image in images:
    # Get the current timestamp for a unique filename
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    value = images.index(image)

    # Construct the save path within your Google Drive folder
    save_path = '/kaggle/working/gens/image_{}_{}.png'.format(timestamp, value)

    # Save the image
    image.save(save_path)

    print(f"Image saved to: {save_path}")

    shareable_link = upload_to_cloudinary(save_path)

    # Call Make.com webhook with the public link
    webhook_url = 'https://hook.us1.make.com/4mkwesvlf0otqopcp9sifqqiwmxs926v'
    requests.post(webhook_url, json={'shareable_link': shareable_link})